In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt
import re

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [3]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [4]:
lambdas = np.array([10,2.5])


In [5]:
folder_path = rf"../Structures"
project_name = "20251028_LSU_Mean_Free_Path_ls"
postprocess_results = []
runtime_ps = 25e-12
min_steps_per_lambda = 20
cuts=[2]
h5_bg = None
ref = True
for direction in ["z"]: 
    for dirpath, dirnames, filenames in os.walk(folder_path):
        try:
            for filename in filenames:
                if filename.endswith(".h5"):
                    ff = float(re.search(r'ff_([+-]?\d+(?:\.\d+)?)', filename).group(1))
                    n_value = float(re.search(r'n_([+-]?\d+(?:\.\d+)?)', filename).group(1))
                    n_eff = np.sqrt((1-ff)+ff*(3.3)**2)

                    for cut in cuts:
                        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
                            continue 
                        if os.path.isfile(os.path.join(dirpath, filename)):
                            file=os.path.join(dirpath, filename)
                            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                                            ,direction=direction, lambda_range=lambdas,
                                                            box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                                           scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                                           monitors=["flux"], flux_monitor_position=18,cell_size_manual=45,sim_bg=n_eff**2,
                                                           freqs=400, 
                                                           cut_condition=cut, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
                                                           )
                            sim=structure_1.sim
                            monitor_field = td.FieldMonitor(
                                    center=[0.0, 0.0, 0.0],
                                    size=[
                                            structure_1.t_slab_x if structure_1.direction == "y" else td.inf,
                                            0,
                                            structure_1.t_slab_z if structure_1.direction == "z" else td.inf
                                        ],
                                    fields=["Ex","Ey","Ez"],
                                    name="field_monitor",
                                    freqs =structure_1.monitor_freqs,
                                        )
                            sim=sim.copy(update={"monitors":list(sim.monitors) + [monitor_field]})
                            folder_desc = rf"H:\phd stuff\tidy3d\data\{project_name}\n_{n_value:.2f}"
                            os.makedirs(folder_desc, exist_ok=True)
                            sim_name=rf"LSU_{Path(filename).stem}_size_{cut}"
                            if os.path.exists(os.path.join(folder_desc, sim_name+".txt")):
                                print("Exist!")
                            else:
                                id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
                                ids = '\n' + id
                                with open(os.path.join(folder_desc, sim_name+".txt"), "w") as file:
                                    # Write the string to the file
                                    file.write(ids)
                                web.start(task_id = id)
                                web.monitor(id)
                            

        except Exception as e:
            print(f"Error processing {dirpath}: {e}")
        
    



Configured successfully.
Exist!
Configured successfully.


14:12:47 W. Europe Standard Time Created task 'LSU_n_3.30_ff_0.3107_size_2' with
                                 task_id                                        
                                 'fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85' and
                                 task_type 'FDTD'.

                                 View task using web UI at                      
                                 ]8;id=460926;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=183093;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\task]8;;\
                                 ]8;id=183093;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\Id]8;;\]8;id=460926;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\=]8;;\]8;id=235653;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\fdve]8;;\]8;id=460926;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85']8;;\.

                                 Task folder: ]8;id=31399;https://tidy3d.simulation.cloud/folders/folder-c15f57af-e020-4baa-9ced-c1b586719970\'20251028_LSU_Mean_Free_Path_ls']8;;\.

Output()

...

Output()

Output()

14:19:21 W. Europe Standard Time status = postprocess

14:20:07 W. Europe Standard Time status = success

14:20:09 W. Europe Standard Time View simulation result at                      
                                 ]8;id=564802;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=596354;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\task]8;;\
                                 ]8;id=596354;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\Id]8;;\]8;id=564802;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\=]8;;\]8;id=861975;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\fdve]8;;\]8;id=564802;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85\-1b853cf2-0bb9-408e-8e3a-9cc5b9d76d85']8;;\.